
# Create your Chat Bot App on Databricks!
Are you interested in making a chatbot that can make use of your own collections of data when answering questions? Retrieval-augmented generation (RAG) is an framework that combines the strengths of pre-trained language models and information retrieval systems to generate responses in a conversational AI system or to create content by leveraging external knowledge. It integrates the retrieval of relevant information from a knowledge source and the generation of responses based on that retrieved information.

In today's workshop, we'll create a chatbot application that answers specific questions based on a .pdf file. We'll leverage a few open source libraries to help us do so, and Databricks Foundation Models (Llama-2).

<p></p>

**Process:**
1. **Select the .pdf file you want to use**: You can choose between 3 example .pdf files or load your own.
2. **Process the .pdf file into Databricks vector search**: We'll show you how to open a .pdf file on Databricks, parse the text and split it into chunks. Chunks will then be embedded as vectors into Vector Search Index.
3. **Prepare your Chatbot with LangChain**: Use Langchain and Llama-2 to create a chatbot that will retrieve the text from Databricks Vector Search and answer your .pdf specific questions.
4. **Plug it to Gradio App**: Finally, we wrap the Chatbot into a Gradio app, so you can share it and have a cool interface to use it.


**Requirements:**
- Change your cluster to the runtime **DBR 13.3 LTS ML (Machine Learning)** to have the right environments packages installed.

## Preparing the environment
First, we install the libraries we'll need to use for the Chatbot development.

In [0]:
%pip install "git+https://github.com/mlflow/mlflow.git@gateway-migration" lxml==4.9.3 transformers==4.30.2 langchain==0.0.344 databricks-vectorsearch==0.22
%pip install -U pypdf gradio
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Cloning https://github.com/mlflow/mlflow.git (to revision gateway-migration) to /tmp/pip-req-build-s0wd5p1q
  Running command git clone --filter=blob:none --quiet https://github.com/mlflow/mlflow.git /tmp/pip-req-build-s0wd5p1q
  Running command git checkout -b gateway-migration --track origin/gateway-migration
  Switched to a new branch 'gateway-migration'
  Branch 'gateway-migration' set up to track remote branch 'gateway-migration' from 'origin'.
  Resolved https://github.com/mlflow/mlflow.git to commit db1e9e851f873aefe2bdc4938a316d665e67fed2
  Running command git submodule update --init --recursive -q
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metad

In [0]:
catalog = "hk_sg_population_2023" ### Rename it to your own catalog name
database = "chatbot"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog}.{database}")
spark.sql(f"USE {catalog}.{database}")

DataFrame[]

## Preparing Databricks Vector Search
Building a Vector Search API from a .pdf file involves a multi-step process. 
- First, we extract relevant data from the PDF using langchain.document_loaders library to convert the .pdf content into a usable format. 
- Next, we'll split the data into constant lenghts - this process is called **chunking**. 
- Given an embedding model hosted on Databricks, we then transform all text chunks into vectors that we can finally store into chromadb.

Let's start with loading the .pdf file from it's Volume location.
You can find various files in the catalog chatbot > resources > resource_name. 

Feel free to also add your own for extra points!


In [0]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader,PyPDFLoader,DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
import pandas as pd
import os

In [0]:
pdf_path = "/Volumes/dbdemos/chatbot/hk_sg_population_2023/population-in-brief-2023.pdf"
display(dbutils.fs.ls(pdf_path))

path,name,size,modificationTime
dbfs:/Volumes/dbdemos/chatbot/hk_sg_population_2023/population-in-brief-2023.pdf,population-in-brief-2023.pdf,1425841,1702364456000


### Document Chunking
We load the .pdf file and split it into chunk of 1000 characters size. 

Each chunks overlaps each other of 200 characters to avoid losing information between cuts.

In [0]:
loader = DirectoryLoader("/Volumes/dbdemos/chatbot/hk_sg_population_2023/", glob="population-in-brief-2023.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

29

In [0]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

Let's see how a chunk looks like.

In [0]:
texts[1]

Document(page_content='Population in Brief 20232Contents\nOverview\nKey Indicators\nOverall Population\nAgeing\nMarriage & Parenthood\nImmigration & Citizens By Descent04\n05\n06\n09\n12\n16', metadata={'source': '/Volumes/dbdemos/chatbot/hk_sg_population_2023/population-in-brief-2023.pdf', 'page': 1})

### Embedding Model
Now we load an embedding model to vectorise these chunks. The model used is the **bge-large-en** from our Foundational Models feature. It maps sentences & paragraphs to a 1024 dimensional dense vector space and can be used for tasks like clustering or semantic search.

In [0]:
import mlflow.deployments

# Let's try our embedding model here with some input
deploy_client = mlflow.deployments.get_deploy_client("databricks")
response = deploy_client.predict(endpoint="databricks-bge-large-en", inputs={"input": ["What is the sound system?"]})
embeddings = [e["embedding"] for e in response.data]
print(embeddings)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-c2bf809e-2370-4ddb-8a4c-e027bc10fb33/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


[[-0.03814697265625, 0.0189971923828125, -0.04425048828125, 0.01097869873046875, -0.0186004638671875, -0.055023193359375, -0.05145263671875, -0.0133819580078125, 0.0234375, 0.0225067138671875, 0.009002685546875, 0.03424072265625, 0.00397491455078125, 0.003780364990234375, -0.0157928466796875, 0.00298309326171875, -0.0699462890625, 0.0187225341796875, -0.07379150390625, -0.0194244384765625, -0.01256561279296875, 0.004512786865234375, -0.0089874267578125, 0.00616455078125, -0.042388916015625, -0.0027484893798828125, -0.0289764404296875, 0.05084228515625, 0.00702667236328125, 0.04815673828125, -0.007266998291015625, -0.01470947265625, 0.022918701171875, -0.0267333984375, 0.0111236572265625, -0.053253173828125, -0.003208160400390625, -0.047454833984375, -0.0249786376953125, -0.07781982421875, 0.030242919921875, 0.0020236968994140625, -0.0099639892578125, -0.03265380859375, -0.0218963623046875, -0.0021686553955078125, 0.041839599609375, -0.024932861328125, 0.04339599609375, 0.02130126953125


### Compute embeddings using BGE Embedding Model endpoint

We first split the contents list into smaller batches of 150 items each, in order to respect the maximum number of inputs that the embedding model takes per request.

For each batch of contents, the function sends a predict request to the Databricks BGE endpoint, passing as input a dictionary that maps the "input" key to the batch contents. We then collect the corresponding embeddings from the response as a list and concatenates them with any previous embeddings obtained from other batches.


In [0]:
def get_embedding(contents):
    def get_embeddings(batch):
        #Note: this will gracefully fail if an exception is thrown during embedding creation (add try/except if needed) 
        response = deploy_client.predict(endpoint="databricks-bge-large-en", inputs={"input": batch})
        return [e['embedding'] for e in response.data]

    # Splitting the contents into batches of 150 items each, since the embedding model takes at most 150 inputs per request.
    max_batch_size = 150
    batches = [contents[i:i + max_batch_size] for i in range(0, len(contents), max_batch_size)]

    # Process each batch and collect the results
    all_embeddings = []
    for batch in batches:
        all_embeddings += get_embeddings(batch)

    return pd.Series(all_embeddings)

contents = [text.page_content for text in texts]
embeddings = get_embedding(contents)

### Saving the embeddings to our Delta Table

Note that this part would typically be setup as a production-grade job, running as soon as a new documentation page is updated. <br/> This could be setup as a Delta Live Table pipeline to incrementally consume updates.

In [0]:
# Convert texts array into a Spark DataFrame
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType
from pyspark.sql.functions import col, monotonically_increasing_id

schema = StructType([
    StructField("page_content", StringType(), True),
    StructField("metadata_source", StringType(), True),
    StructField("page", StringType(), True),
    StructField("embedding", ArrayType(FloatType()), True)
])

data = [(text.page_content, text.metadata["source"], text.metadata["page"], embedding) for text, embedding in zip(texts, embeddings)]
df = spark.createDataFrame(data, schema=schema).withColumn("id", monotonically_increasing_id())

In [0]:
# Write the Spark Dataframe into a Delta Table
table_name = "hk_sg_population_2023" ### Feel free to change the name 

df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable(table_name)
spark.sql(f"ALTER TABLE {table_name} SET TBLPROPERTIES ('delta.enableChangeDataFeed' = 'true')")
display(spark.table(table_name))

page_content metadata_source page embedding id Population in Brief 202321
Annex
Residents 38.9 40.8 42.1 42.4
Citizens 40.0 41.7 42.8 43.0
25 - 29 years 84.4 80.7 84.3 82.5
30 - 34 years 45.9 40.5 47.0 42.5
35 - 39 years 23.5 24.2 25.8 25.5
40 - 44 years 15.3 17.0 18.4 18.5
45 - 49 years 13.7 13.4 14.6 14.9
25 - 29 years 68.5 68.1 71.7 71.4
30 - 34 years 33.2 32.8 37.4 34.6
35 - 39 years 20.2 20.8 21.8 20.8
40 - 44 years 14.3 18.1 21.6 19.0
45 - 49 years 13.9 15.9 18.8 17.22013
Proportion of singles among citizens males by age group
Proportion of singles among citizen females by age group20122018
20172022
20212023
2022Table 6: Median age, as at June (years)
Table 7: Proportion of singles among citizen males / females aged 25-49 years 
by age group, as at June (%)Source: Department of Statistics
Source: Department of Statistics /Volumes/dbdemos/chatbot/hk_sg_population_2023/population-in-brief-2023.pdf 20 List(0.0065307617, 0.025146484, -0.01600647, 0.0028057098, -0.03390503, -0.01524353, 0.005958557, 0.02230835, 0.04296875, 0.05130005, 0.029296875, -0.045928955, -0.010986328, 0.015342712, -0.028869629, 0.029434204, -0.030776978, -0.018203735, -0.012413025, 0.014053345, -0.035888672, 0.029754639, -0.03744507, -0.047698975, -0.02909851, 0.028579712, 0.004142761, 0.02758789, 0.02368164, 0.018844604, -0.007835388, -0.056854248, 0.018707275, -0.036895752, 0.03125, 0.02029419, 0.023742676, -0.039245605, 0.048980713, -0.019683838, 0.0038928986, -0.026885986, 0.027130127, 0.013648987, -0.04876709, 4.4846535E-4, -0.0065956116, -0.036987305, 0.013374329, 0.0027103424, -2.6524067E-5, -2.8133392E-4, 0.038024902, -0.013343811, 0.039855957, 0.023635864, -0.011566162, -0.019805908, -0.005596161, 0.03677368, -0.02482605, 0.037719727, 0.04095459, -0.03161621, 9.7453594E-5, 0.06762695, 0.015396118, -0.032592773, 0.018844604, -0.044677734, 0.015342712, -0.04058838, -0.07574463, -0.0061683655, -0.047546387, 0.013046265, 0.026229858, 0.0040626526, -0.015701294, 0.014595032, 0.015556335, 0.006034851, 0.009101868, -0.02319336, -0.008132935, -0.008384705, 0.012397766, 0.034973145, 0.009422302, -1.2052059E-4, 0.019638062, 0.040771484, -0.009025574, -0.02645874, -0.0104522705, 0.009048462, -0.031082153, -0.0039787292, -0.0021133423, -0.041778564, 0.04171753, 0.023101807, 7.686615E-4, 0.059448242, -0.023452759, 0.020477295, -0.029785156, 0.015777588, 0.030395508, -0.028625488, 0.028320312, 0.007881165, 0.019561768, 0.01576233, 0.0256958, 0.070739746, -0.01184845, 0.008155823, -0.07696533, 0.009613037, -0.0018701553, 0.033050537, 0.059509277, -0.026428223, 0.014862061, -0.04272461, 0.0025844574, 0.028930664, -0.031799316, -0.013038635, -0.04046631, -0.015327454, 0.042877197, 0.011672974, 0.008644104, 0.03942871, 0.019836426, 0.034698486, 0.032196045, -0.005592346, -0.013877869, 0.03125, -0.025009155, 0.06890869, -0.028198242, 0.05618286, -0.03845215, -0.03277588, -0.013008118, 0.0073165894, -0.017089844, -0.00178051, -0.021347046, 0.021606445, -0.019378662, -0.007446289, -0.019683838, 0.028900146, -0.006462097, -0.0027008057, -0.046417236, -0.032470703, 0.032836914, 0.024978638, -0.05267334, 0.015686035, -0.024475098, 0.011520386, 0.0038700104, -0.011268616, 0.048736572, -0.018585205, -0.009841919, 0.005962372, 0.03805542, 0.05206299, -0.004108429, -0.03656006, 0.035491943, 0.008148193, -0.02067566, -5.3048134E-6, 0.014099121, -6.7424774E-4, 0.017059326, 0.03378296, -0.024230957, -0.04019165, -0.019195557, -0.043640137, -0.012916565, 0.04083252, -0.0064735413, 0.017852783, 4.2700768E-4, -0.011566162, -0.012130737, 0.020187378, -0.011764526, -0.028564453, -0.03643799, 0.030715942, 4.374981E-4, 0.05871582, 0.0042037964, -0.007873535, 0.008674622, 0.0635376, -0.0011024475, 0.022644043, 0.05105591, -0.0011320114, -0.022583008, 0.025161743, 0.048217773, 0.014434814, -0.012107849, 0.0073547363, -0.029693604, 0.0044822693, 0.025543213, 0.03062439, 0.0037975311, -0.012710571, 0.04043579, -0.0052833557, 0.002216339, 0.044525146, -0.020065308, 0.058532715, 0.01759


### Our dataset is now ready! Let's create our Self-Managed Vector Search Index.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-self-managed-prep-3.png?raw=true" style="float: right; width: 600px; margin-left: 10px">

Our dataset is now ready. We chunked the documentation page into small sections, computed the embeddings and saved it as a Delta Lake table.

Next, we'll configure Databricks Vector Search to ingest data from this table.

Vector search index uses a Vector search endpoint to serve the embeddings (you can think about it as your Vector Search API endpoint). <br/>
Multiple Indexes can use the same endpoint. Let's start by creating one.

In [0]:
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.catalog as c
from databricks.vector_search.client import VectorSearchClient

# Use an existing vector search API endpoint
vsc = VectorSearchClient(disable_notice=True)
VECTOR_SEARCH_ENDPOINT_NAME="dbdemos_vs_endpoint"

# The table we'd like to index
source_table_fullname = f"{catalog}.{database}.{table_name}"

# Where we want to store our index
vs_index_fullname = f"{catalog}.{database}.{table_name}_index"

print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
vsc.create_delta_sync_index(
  endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
  index_name=vs_index_fullname,
  source_table_name=source_table_fullname,
  pipeline_type="CONTINUOUS",
  primary_key="id",
  embedding_dimension=1024, # Match your model embedding size (bge)
  embedding_vector_column="embedding"
)

Creating index hk_sg_population_2023.chatbot.hk_sg_population_2023_index on endpoint dbdemos_vs_endpoint...


Now you've persisted and created your vector search on Databricks :) ! 

/!\ Before using the Vector Search API, you might need to wait for the deployment to complete. It usually takes about **one minute**. 

You can check the status of your index deployment here: [Vector Search Endpoint](https://dbc-07a52045-ffe9.cloud.databricks.com/?o=1346073101933532#setting/clusters/vector-search/dbdemos_vs_endpoint)

### Test the Vector Search index
We now can leverage our Vector Search index for similarity search. Simply submit queries and retrieve the closest matches within the database based on predefined similarity metrics.

Let's package a `retriever` function to get most similar documents from our vector search given an query. 

Here, for ease of the workshop, we use user authentification token loaded from the notebook. In production workload, we'd use Service Principal for authentification instead.

In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain.vectorstores import DatabricksVectorSearch
from langchain.embeddings import DatabricksEmbeddings

# Embedding model and API configurations
embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")
host = "https://dbc-07a52045-ffe9.cloud.databricks.com"
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

def get_retriever(persist_dir: str = None):
    os.environ["DATABRICKS_HOST"] = host
    
    #Get the vector search index
    vsc = VectorSearchClient(workspace_url=host, personal_access_token=token, disable_notice=True)
    vs_index = vsc.get_index(
        endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
        index_name=vs_index_fullname
    )

    # Create the retriever
    vectorstore = DatabricksVectorSearch(vs_index, text_column="page_content", columns=["page", "metadata_source"], embedding=embedding_model)
    return vectorstore.as_retriever()

In [0]:
# test our retriever
vectorstore = get_retriever()
similar_documents = vectorstore.get_relevant_documents("What is the overall singapore population?")
print(f"Relevant documents: {similar_documents[0]}")

Relevant documents: page_content='Population in Brief 20236Overall Population\nAs at June 2023, there were 3.61 million Singapore Citizens (SCs) and 0.54 million Permanent \nResidents (PRs). In total, there were 4.15 million residents.1 Non-residents (NRs), which comprise our \nforeign workforce across all pass types, dependants and international students, totalled 1.77 million. \nOverall, Singapore’s total population stood at 5.92 million.\nSingapore’s Total Population Stood at 5.92M\nDiagram 1: Total population, as at June 20232\nSource: Department of Statistics, Ministry of Manpower\nFigures may not sum up due to rounding.\n1 Singapore residents comprise SCs and PRs.\n2 The figures are as at end-June 2023.\nWork Permit Holders are semi-skilled foreign workers from approved source countries. Some work in Con struction, Marine Shipyard and Process \n(CMP) sectors (e.g. construction workers), and others work in non-CMP sectors (e.g. Services, Manufactu ring).' metadata={'page': '5', 'm

## Preparing the Chatbot with LangChain and OpenAI
Langchain offers a powerful solution for creating a document retrieval chatbot by employing advanced natural language processing techniques. With Langchain, developers can seamlessly integrate document retrieval capabilities, plug an existing LLM model to enable efficient search, extraction and response generation to users queries.

This Chatbot will be powered by OpenAI API, and we'll be using text-davinci-003 model. We'll start by setting up the OpenAI model by passing the API keys from Databricks secrets and loading the endpoint.

In [0]:
# Test Databricks Foundation LLM model
from langchain.chat_models import ChatDatabricks
chat_model = ChatDatabricks(endpoint="databricks-llama-2-70b-chat", max_tokens = 200)
print(f"Test chat model: {chat_model.predict('How much is Singapore Population in 2023?')}")

Test chat model: 
As of February 2023, the estimated population of Singapore is around 5.69 million people. The population of Singapore has been steadily increasing over the years, with a growth rate of around 1-2% per year. However, the government has implemented measures to slow down the population growth, such as stricter immigration policies and family planning initiatives, in order to manage the country's resources and infrastructure.


Now we can package with LangChain our Chatbot powered by Llama-2-70b that retrieves documents from our vector search retriever, and answer the user's query.

In [0]:
qa_chain = RetrievalQA.from_chain_type(llm=chat_model, 
                                  chain_type="stuff", 
                                  retriever=vectorstore, 
                                  return_source_documents=True)

Let's test it with a query :)

In [0]:
query = "What about non-citizen population in singapore?" ### You can do further prompt engineering here :)
llm_response = qa_chain(query)
display(pd.DataFrame(llm_response))

/databricks/spark/python/pyspark/sql/pandas/conversion.py:401: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Could not convert Document(page_content='Population in Brief 20236Overall Population\nAs at June 2023, there were 3.61 million Singapore Citizens (SCs) and 0.54 million Permanent \nResidents (PRs). In total, there were 4.15 million residents.1 Non-residents (NRs), which comprise our \nforeign workforce across all pass types, dependants and international students, totalled 1.77 million. \nOverall, Singapore’s total population stood at 5.92 million.\nSingapore’s Total Population Stood at 5.92M\nDiagram 1: Total population, as at June 20232\nSource: Department of Statistics, Ministry of Manpower\nFigures may not sum up due to rounding.\n1 Singapore residents comprise SCs and PRs.\n2 The figures are as at end-June 2023.\nWork Permit Holders are semi-skilled foreign 

query,result,source_documents
What about non-citizen population in singapore?,"According to the information provided, as of June 2023, the non-resident population in Singapore stood at 1.77 million, an increase of 13.1% from June 2022. This increase was mainly due to foreign employment growth in Singapore from June 2022 to June 2023, with the largest increase coming from Work Permit Holders in the Construction, Marine Shipyard, and Process sectors. The remaining increases in foreign employment were spread across sectors as firms backfilled positions vacated by non-residents during the pandemic.","List(Map(metadata_source -> /Volumes/dbdemos/chatbot/hk_sg_population_2023/population-in-brief-2023.pdf, id -> 9.0, page -> 5), Population in Brief 20236Overall Population As at June 2023, there were 3.61 million Singapore Citizens (SCs) and 0.54 million Permanent Residents (PRs). In total, there were 4.15 million residents.1 Non-residents (NRs), which comprise our foreign workforce across all pass types, dependants and international students, totalled 1.77 million. Overall, Singapore’s total population stood at 5.92 million. Singapore’s Total Population Stood at 5.92M Diagram 1: Total population, as at June 20232 Source: Department of Statistics, Ministry of Manpower Figures may not sum up due to rounding. 1 Singapore residents comprise SCs and PRs. 2 The figures are as at end-June 2023. Work Permit Holders are semi-skilled foreign workers from approved source countries. Some work in Con struction, Marine Shipyard and Process (CMP) sectors (e.g. construction workers), and others work in non-CMP sectors (e.g. Services, Manufactu ring)., Document)"
What about non-citizen population in singapore?,"According to the information provided, as of June 2023, the non-resident population in Singapore stood at 1.77 million, an increase of 13.1% from June 2022. This increase was mainly due to foreign employment growth in Singapore from June 2022 to June 2023, with the largest increase coming from Work Permit Holders in the Construction, Marine Shipyard, and Process sectors. The remaining increases in foreign employment were spread across sectors as firms backfilled positions vacated by non-residents during the pandemic.","List(Map(metadata_source -> /Volumes/dbdemos/chatbot/hk_sg_population_2023/population-in-brief-2023.pdf, id -> 1.7179869185E10, page -> 15), Population in Brief 202316Immigration & Citizens by Descent Immigration helps to moderate the impact of ageing and low birth rates on our citizen population, and keeps it from shrinking over the longer term. Singapore takes in a carefully controlled number of PRs and new citizens each year. New citizenships are granted to individuals who can integrate and contribute to Singapore, and are committed to making Singapore their home. New citizens either share family ties with Singaporeans (for example, through marriage), or have studied, worked or lived in Singapore for some time. All new adult citizens come from our pool of PRs, as individuals have to obtain permanent residence before they may apply for citizenship in Singapore. The small group of new citizens who were not PRs prior to obtaining citizenship were minors who were either (i) children of citizens, including those born overseas to Singaporean parents, or (ii) children of PRs who were included as dependants in, Document)"
What about non-citizen population in singapore?,"According to the information provided, as of June 2023, the non-resident population in Singapore stood at 1.77 million, an increase of 13.1% from June 2022. This increase was mainly due to foreign employment growth in Singapore from June 2022 to June 2023, with the largest increase coming from Work Permit Holders in the Construction, Marine Shipyard, and Process sectors. The remaining increases in foreign employment were spread across sectors as firms backfilled positions vacated by non-residents during the pandemic.","List(Map(metadata_source -> /Volumes/dbdemos/chat

Cool work! Our query gets a response for each similar documents retrieved. We can see the related documents pieces that helped the model generate an answer. 

Now, how can we beautify the usage of this chatbot, and share it to our team members?


## Creating the App interface using Gradio

Gradio simplifies rapid application development in Python notebooks by offering an intuitive interface for creating interactive user interfaces with minimal code. By effortlessly integrating pre-built UI components, developers can easily transform their Python functions or models into interactive apps, making it accessible even to those with limited frontend experience. 

Let's create our gradio app for this chatbot!

In [0]:
import gradio as gr

# You can change the picture path to your corresponding one
image_path = '/Volumes/dbdemos/chatbot/hk_sg_population_2023/hk_cover.png'

def question_answer(question, image):
    output = qa_chain(question)
    print(output)
    doc_pages = ','.join(set([str(doc.metadata['page']) for doc in output['source_documents']]))
    doc_name = ','.join(set([str(doc.metadata['metadata_source']) for doc in output['source_documents']]))
    doc_content = '.\n'.join(set([str(doc.page_content) for doc in output['source_documents']]))
    return output['result'], doc_pages, doc_name, doc_content

gr.Interface(fn=question_answer,
    title="Ask about Singapore Population 2023", 
    inputs=["text",
            gr.Image(value=image_path,height=200)], 
    outputs=[gr.Textbox(label='Chatbot Response'),
         gr.Textbox(label='Document Page Number'),
         gr.Textbox(label='Document Location'),
         gr.Textbox(label='Document Context')],
    theme=gr.themes.Soft()).launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://a4a2816e45de0975d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


You can share this gradio app using the public URL to your colleagues and friend so they can test your chatbot. Congratulations!!